In [1]:
import numpy as np
import os
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import PIL

import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

#model
from tensorflow.keras import layers,models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


#this code is to check if tensorflow is using CPU or CPU
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.14.0
[]


In [2]:

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv'
train_df = train_df.astype(str)
test_df = test_df.astype(str)
test_df = test_df[['image_id']]
train_df['id']=train_df['id'].astype(str)+'.jpg'
test_df['image_id']=test_df['image_id'].astype(str)+'.jpg'

In [3]:
src_path_train = "C:/Users/kingc/Documents/DPL302m/2023falldpl30xm/file/train" #soure of train folder

src_path_test = "C:/Users/kingc/Documents/DPL302m/2023falldpl30xm/file/test" #source of test folder
batch_size = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        #rotation_range=20, #because flood image usually have water in the botton of the image so rotation wont be necessary
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',
        validation_split=0.2)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=src_path_train,
    x_col="id",
    y_col="label",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=42
)
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=src_path_train,
    x_col="id",
    y_col="label",
    target_size=(224, 224),
    batch_size=batch_size,
    subset='validation',
    class_mode='binary',
    shuffle=True,
    seed=42
)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=src_path_test,
    x_col="image_id",
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

Found 4212 validated image filenames belonging to 2 classes.


c:\Users\kingc\miniconda3\envs\tf-gpu\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 15 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


Found 1053 validated image filenames belonging to 2 classes.
Found 1318 validated image filenames.


c:\Users\kingc\miniconda3\envs\tf-gpu\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 15 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(
c:\Users\kingc\miniconda3\envs\tf-gpu\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    'best_model_checkpoint2.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [5]:

from lion_tf import Lion #Lion only support tf version >=2.11. Unfortunately, tf GPU only support up to tf 2.10 so CPU will be used

In [ ]:
backbone =EfficientNetV2XL(weights = 'imagenet-21k-ft1k',include_top=False,input_shape=(224,224,3))
for layer in backbone.layers:
    layer.trainable = False
model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.summary()
model.compile(loss='binary_crossentropy',optimizer= Lion(), metrics=['accuracy'])

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2]

In [12]:
model.fit_generator(train_generator,
                    validation_data = valid_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    callbacks=[checkpoint],
                    epochs=3)

C:\Users\kingc\AppData\Local\Temp\ipykernel_12476\1994012995.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/16
131/131 [==============================] - ETA: 0s - loss: 0.3295 - accuracy: 0.8615
Epoch 1: val_accuracy did not improve from 0.87844
131/131 [==============================] - 1532s 12s/step - loss: 0.3295 - accuracy: 0.8615 - val_loss: 0.2975 - val_accuracy: 0.8689
Epoch 2/16
131/131 [==============================] - ETA: 0s - loss: 0.3284 - accuracy: 0.8557
Epoch 2: val_accuracy did not improve from 0.87844
131/131 [==============================] - 1465s 11s/step - loss: 0.3284 - accuracy: 0.8557 - val_loss: 0.2928 - val_accuracy: 0.8718
Epoch 3/16
131/131 [==============================] - ETA: 0s - loss: 0.3259 - accuracy: 0.8531
Epoch 3: val_accuracy improved from 0.87844 to 0.88604, saving model to best_model_checkpoint2.h5
131/131 [==============================] - 1480s 11s/step - loss: 0.3259 - accuracy: 0.8531 - val_loss: 0.2884 - val_accuracy: 0.8860
Epoch 4/16
131/131 [==============================] - ETA: 0s - loss: 0.3216 - accuracy: 0.8665 
Epoch 4: val_

KeyboardInterrupt: 

In [6]:
from tensorflow.keras.models import load_model

In [7]:
best_score = load_model('best_model_checkpoint_v1.h5', compile=False)


In [8]:
prediction=best_score.predict(test_generator)

1318/1318 [==============================] - 664s 497ms/step


In [11]:
#data processing techniquest to merge the label variable and the test.csv 
img_list = []
for i in test_generator.filenames:
    img_list.append(i)
predict_classes = (prediction > 0.5).astype(int).flatten()
data = {"id":img_list,'label':predict_classes}
df = pd.DataFrame(data)
df = df.replace('.jpg','',regex=True)
df.to_csv("submission_demo.csv",index=False)
df_test = pd.read_csv("test.csv")
df = pd.read_csv("submission_demo.csv")
df_test.rename(columns={'image_id':'id'},inplace=True)
df_test = df_test['id']
df_final = pd.merge(df_test,df,on='id',how='left')
df_final['label'].fillna("0",inplace=True)
df_final.isnull().values.any()
df_final['label'] = df_final['label'].astype(int)
df_final

,id,label
0,3483809003,1
1,3712805295,0
2,379845620,0
3,7343264988,1
4,3843337492,1
...,...,...
1315,6452132743,0
1316,244899140,0
1317,3073018258,1
1318,49525361,0


In [10]:
df_final.to_csv('C:/Users/kingc/Documents/DPL302m/2023falldpl30xm/file/submission/submission_final.csv',index=False) 
#export test.csv file with the labels